##Red neuronal para predicción de alquiler de bicicletas

Construiremos una red neuronal, esta vez para un problema de regresión: predicción de la cantidad de bicicletas alquiladas, según el conjunto de datos *bikeshare_hour*, el cuál se proveerá. Adicionalmente, en esta ocasión encontraremos el mejor modelo usando un conjunto de validación.

Cargar en formato .ipynb o .html en aulas a más tardar el día lunes 17 de marzo.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data 
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

### Cargar y preparar los datos

Un paso muy importante en redes neuronales es preparar correctamente los datos. Variables con diferentes escalas le dificulta a la red aprender eficientemente los pesos correctos. 


In [ ]:
df=pd.read_csv('bikeshare_hour.csv')
df.head()

Este dataset contiene el número de alquileres para cada hora de cada día desde Enero 1 de 2011 hasta Diciembre 31 de 2012. El número de usuarios que alquilaron se divide en regitrados *(registered)* y casuales *(casual)*, los cuales se suman en la columna *cnt*, la cuál será nuestra variable objetivo.


Los fines de semana tienen un número más bajo de alquileres y hay picos cuando las personas se dirigen desde y hacia el trabajo durante la semana También tenemos información acerca de la temperatura, humedad, velocidad del viento, todas estas afectando el npumero de alquileres. Trataremos de capturar esta información con nuestro modelo.



In [ ]:
df[:24*10].plot(x='dteday', y='cnt')

Hay algunas variables categóricas: **season**, **weathersit**, **mnth**, **hr**, **weekday**. Nevesitamos crear variables dummy para éstas. 

Así mismo, eliminaremos algunas variables que redundan o no aportan para el modelamiento el modelamiento: **instant**, **dteday**, **atemp**, **workingday**,**registered**, **casual** así como las **columnas originales de las variables que convertimos en dummies** si es necesario.

In [ ]:
# TO_DO1 Crear las variables dummies indicadas, y eliminar los atributos indicados

#### Cambiar la escala en variables cuantitativas (normalizar).
Para que el entrenamiento sea más fácil, estandarizaremos los datos de las variables contínuas, de manera que tengan media 0 y desviación estandar 1. 

Para esto, en las columnas de variables contínuas usamos la media y desviación estandar de la respectiva columna

$$col:=\frac{col-mean}{standardeviation}$$ 

In [ ]:
# TO_DO 2 Estandarizar las columnas cuantitativas: cnt, temp, hum, windspeed.

#### Crear conjuntos de entrenamiento, validación y test

En esta ocasión seleccionaremos los conjuntos de entrenamiento, vlaidación y test de forma ordenada. Para el test, seleccionaer los datos de aproximadamente los últimos 21 días. Para el conjunto de validación, tomar los datos de aproximadamente los últimos 60 días de los datos restantes.

In [ ]:
# TO_DO3 Definir los conjuntos de train, test y validation. Darles los nombres train, val, test.

#### Convertir los datos a tensores y prepararlos para alimentar la red
A continuación  crearemos una clase (*MyDataset*) que nos prepara los datos para alimentar la red neuronal, convirtiendolos a parejas ordenadas de tensores conteniento los atributos y la variable objetivo. Sus parámetros son: el dataset df y el nombre de la columna objetivo en el dataset.


In [ ]:
class MyDataset():
 
  def __init__(self,df,target_column):
    #price_df=pd.read_csv(file_name)
 
    #x=price_df.iloc[:,0:8].values
    #y=price_df.iloc[:,8].values
    # y_train = df_train['activity']
    # X_train = 
    y=df[target_column].values
    X=df.drop(target_column,axis=1).values
    self.X=torch.tensor(X,dtype=torch.float32)
    self.y=torch.tensor(y,dtype=torch.float32)
 
  def __len__(self):
    return len(self.y)
   
  def __getitem__(self,idx):
    return self.X[idx],self.y[idx]

Ahora usamos los Dataloaders para los conjuntos set, val y test.

In [ ]:
# Usar la clase MyDataset para preparar cada conjunto en forma de tensores
train_sec=MyDataset(train,'cnt')
test_sec=MyDataset(test,'cnt')
val_sec=MyDataset(val,'cnt')

In [ ]:
# Definir los DataLoaders para cargar la información pro lotes
train_data=DataLoader(
    train_sec,
    batch_size=1,
    shuffle=False,
    #num_workers=0,
    #collate_fn=None,
    #pin_memory=False,
 )

test_data=DataLoader(
    test_sec,
    batch_size=3,
    shuffle=False,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

val_data=DataLoader(
    val_sec,
    batch_size=3,
    shuffle=False,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

Vamos a imprimir el primer bath del Test Set para visualizar y entender, sus tamaños y cómo el Dataloader ingresan los datos a la red neuronal.

In [ ]:
for i, (data, labels) in enumerate(test_data):
  print(data.shape, labels.shape)
  print(data,labels)
  break;

### Definir la classe Net con la estructura de la red neuronal

Basándose en el taller pasado, construya una red neuronal que tenga las siguientes características: 
- Una sola capa oculta. Usted decida el número de nodos. (Puede hacer entrenamientos pequeños, con una sola epoch por ejemplo, para hacer pruebas y decidir un buen número de nodos. Entre más nodos aprenderá más características de los datos, pero tardará más. Busque un buen equilibrio).
- Una función de activación signoide para la capa oculta.







In [ ]:
# TO_DO 4 Escribir el código para la arquitectura de la red neuonal.

Revisamos que estemos usando GPU y definimos el dispositivo 

In [ ]:
gpu_avail = torch.cuda.is_available()
print(f"Is the GPU available? {gpu_avail}")

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Definimos el modelo, el optimizadoy y la función de costo.

In [ ]:
model=Net()

#TO_DO 5 Definir el optimizador Stochastic gradient descent y la función MeanSquareError. Usar Learnig rate de 0.1
optimizer=
criterion=

### Entrenando la red y guardando el mejor modelo

A continuación definimos la función de entrenamiento.

In [ ]:
#pasamos el modelo al dispositivo GPU
model.to(device)
def train_model(model,optimizer,loss_module,train_loader,valid_loader,num_epochs):
  
  valid_loss_min =np.inf  #Vamos a encontrar el menor valor de error de validación. Por eso la inicializmaos como 'infinito'
  
  for i in range(num_epochs):
    model.train()  #ponemos el modelo en modo entrenamiento. Es importante en otras arquitecturas como redes convolucionales.
    train_loss = 0.0
    v_loss = 0.0

    # TODO 6 Completar el código a continuación
    for data, target in train_loader:
        # mover los tensores de atributos y etiquetas al dispositivo GPU

        # Reiniciar los gradientes

        # forward pass: calcular la salida para los datos de entrada..

        # calculate the batch loss
        
        # backpropagation: cálculo de gradientes
   
        # actualizar los parámetros

        # actualizar la cuenta de costos a lo largo de los lotes
        train_loss += loss.item()*data.size(0)
    # for data,labels in testloader:

    train_loss = train_loss/len(train_loader.dataset) 

    model.eval() #Ponemos el modelo en modo evaluación.




    #for param in model.parameters():
    #  print(param.data)
    # vamos a evaluar el modelo entrenado, calculando predicciones con el conjunto de validación
    for data,target in valid_loader:
      data=data.to(device)
      target=target.to(device)
      output=model(data)
      valid_loss= criterion(output, target)
      valid_loss += loss.item()*data.size(0)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
    #imprimir estadísticas de entrenamiento y validación
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        i, train_loss, valid_loss))
    

    #Guardamos el modelo con el menor error de validación.
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_bikeshare.pt')
        valid_loss_min = valid_loss




Ahora entrene el modelo. Intente primero con una sola epoch para verificar que el codigo esté correcto. Luego de eso prube con más epochs. Con un buen rato de tiempo disponible podría intentar 100, 500, las que más quiera intentar de acuerdo a como vea su desempeño.

In [ ]:
train_model( )  #completar

Cargamos el mejor modelo obtenido del entrenamiento. 
*Observación:* En el entrenamiento se guarda un modelo en un archivo .pt usted puede descargar el archivo y guardarlo localmente. De esta forma, si quiere usar el modelo nuevamente sin volver a realizar entrenamiento, sólo carguelo como se indica en la siguiente celda.

In [ ]:
model.load_state_dict(torch.load('model_bikeshare.pt'))

Visualizamos los parámetros del modelo obtenido anteriormente

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data)

### Evaluación del modelo
Ahora la prueba final con el test set. 

In [ ]:
test_loss=0.0

criterion= nn.MSELoss()
for data, target in test_data:
  data=data.to(device)
  target=target.to(device)
  output=model(data)
  loss= criterion(output,target)
  test_loss += loss.item()*data.size(0)
test_loss = test_loss/len(test_data.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))


### Usando el modelo en contexto

TO_DO 7: Finalmente, utilice el modelo encontrado para predecir el número de bicicletas rentadas en tres momentos (fecha, hora y demás atributos) diferentes que usted elija (pueden ser tomados del test set o con unos atributos nuevos que usted elija). 
Comente por favor lo siguiente: 
- Escribr si son atributos que usted seleccionó del dataset o unos nuevos.
- Describa cómo obtuvo el valor de la predicción y qué valor dió (tenga en cuenta que la salida está normalizada, pero queremos la cantidad de bicicletas).
- Si son atributos seleccionados del test set, compare su resultado con la etiqueta original. ¿Qué tal se desempeñó en esos tres casos?